Evaluate different ising influence strategies on the Pokec dataset for a range of different linear field gradients.

Created on: 28/09/20

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import ast
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm
import random
import itertools
import matplotlib
from scipy import stats
import math

from ising_block_level_influence import N_Block_sbm_class as NBlock
from ising_block_level_influence import projection_simplex as proj
from ising_block_level_influence import mean_field_IIM
from spatial_spin_monte_carlo import spatial_spin_monte_carlo as Spins
import Pokec_processing as  PokProc

from pokec_utils import *

### Read in processed data

This data has been generated using:

1. **make_Pokec_reduced_profiles.ipynb** - which then feeds data into:

2. The pre-processing script **'make_bratislava_graph_and_blocks.py'**.

In [2]:
graph = nx.read_graphml('Data/Bratislava_graph.graphml')
beta_c = Spins.crit_beta_sparse(graph)
bratislava_profiles = pd.read_csv("Data/bratislava_profiles.csv")
coupling_graph = nx.read_graphml('Data/Bratislava_coupling.graphml')
block_data = pd.read_csv('Data/block_info.csv',converters={'Block' : ast.literal_eval})
mean_block_ages = list(block_data['average_age'])
block_sizes = list(block_data['block_size'])
block_names = list(block_data['block_name'])
block_data['age_group'] = [ 'ages_' + k.split('_')[-1] for k in list(block_data['block_name'])]
block_data.head()

Computing critical temperature


,Unnamed: 0,block_name,block_size,average_age,age_group
0,0,Okolie_ages_1-17,1234,9.0,ages_1-17
1,1,Okolie_ages_18-21,1939,19.5,ages_18-21
2,2,Okolie_ages_22-28,3154,25.0,ages_22-28
3,3,Okolie_ages_29-112,3458,70.5,ages_29-112
4,4,Petrzalka_ages_1-17,1398,9.0,ages_1-17


In [3]:
def linear_field(x : np.ndarray,gradient :float) :
    return gradient*x
#Scale ages to [-1,1]:
rescaled_ages = [ (k-np.mean(mean_block_ages))/(max(mean_block_ages)-min(mean_block_ages)) for k in mean_block_ages ]

In [4]:
bratislava_profiles_indices = bratislava_profiles.reset_index()
groups = [ bratislava_profiles_indices.loc[bratislava_profiles_indices['block']==block] for block in block_names]
groups_node_ids = [list(k['index']) for k in groups]

## Sweep over $\beta$ values

In [5]:
def check_group(input_val,group_label) :
    if input_val == group_label :
        return 1.0
    else :
        return 0.0
    
def mean_and_se(values) :
    return f"{np.mean(values)} +/- {stats.sem(values)} "

#Seed the random number generators:
seed = 1
random.seed(seed)
np.random.seed(seed)

Field_Budget = 2500.0
T=10000
T_Burn = 5*300000
Samples = 15
sample_frac = 1.0 # for snapshot control
eval_data = pd.DataFrame()
control_data = pd.DataFrame()


beta_factor_vals = [8.0]
grad_vals =  np.arange(1.0,11.0,1.0)

for gradient in tqdm.tqdm_notebook(grad_vals) : 
    
    age_field = [linear_field(a,gradient) for a in rescaled_ages ]
    age_field_map = {k:j for k,j in zip(list(block_data['age_group']),age_field)}

    # Block MF setup:
    coupling_graph = nx.convert_node_labels_to_integers(coupling_graph)
    block_graph_ising_system = mean_field_IIM.mean_field_ising_system(coupling_graph,age_field)
    block_graph_ising_system.gamma = 1.0
    block_graph_ising_system.tol = 1E-5
    block_graph_ising_system.max_mf_fp_iterations = 10000
    block_graph_ising_system.mf_fp_init_state = 1.0*np.ones(len(block_sizes))
    block_graph_ising_system.mf_fp_noisy = False

    # Full graph Mf setup
    background_field = np.asarray([age_field_map[k] for k in list(bratislava_profiles['age_group'])])
    relab_graph = nx.relabel.convert_node_labels_to_integers(graph)
    full_graph_ising_system = mean_field_IIM.mean_field_ising_system(relab_graph,background_field)
    full_graph_ising_system.mf_fp_init_state = 1.0*np.ones(len(graph))
    full_graph_ising_system.mf_fp_noisy = False


    for beta_factor in tqdm.tqdm_notebook(beta_factor_vals) : 

        beta = beta_c*beta_factor

        # Block gradient computation: 
        m_block = block_graph_ising_system.mf_magnetization(age_field,beta)
        mag_gradient = block_graph_ising_system.mf_magnetization_gradient(m_block, beta)
        mag_grad_map = { i:j for i,j in zip(block_names,mag_gradient)}
        block_names_list = list(bratislava_profiles['block'])
        block_mf_gradient = np.asarray([mag_grad_map[k] for k in block_names_list])

        # Full gradient computation: 
        m_full = full_graph_ising_system.mf_magnetization(background_field,beta)
        mag_grad_full = full_graph_ising_system.mf_sparse_magnetisation_gradient(m_full, beta)

        # Compute the controls:
        H_full = np.sum(mag_grad_full)
        H_block = np.sum(block_mf_gradient)
        uniform_control = (Field_Budget / len(graph)) * np.ones(len(graph))
        mag_grad_full_at_field_budget = (Field_Budget/H_full)*mag_grad_full
        mag_grad_block_at_field_budget = (Field_Budget/H_block)*block_mf_gradient
        negative_cancelling_field = []
        for field in background_field :
            if field < 0.0:
                negative_cancelling_field.append(-1.0*field)
            else:
                negative_cancelling_field.append(0.0)
        negative_cancelling_field = np.asarray(negative_cancelling_field)
        negative_cancelling_field = (Field_Budget/np.sum(negative_cancelling_field))*np.asarray(negative_cancelling_field) 
        swing_voter_nodes = np.asarray([check_group(k,'ages_22-28') for k in list(bratislava_profiles['age_group']) ])
        H_SV = np.sum(swing_voter_nodes)
        swing_voter_control =  (Field_Budget/H_SV)*swing_voter_nodes
        
        #Snapshot control
        initial_state = 1.0*np.ones(len(graph))
        block_snapshot = Run_MonteCarlo_Snapshot(relab_graph,groups_node_ids ,0, beta_factor,beta_c ,T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=background_field,initial_state=initial_state,frac_to_sample=sample_frac)
        snapshot_gradient = [ (1.0-k**2) for k in block_snapshot ] 
        snapshot_grad_map = { i:j for i,j in zip(block_names,snapshot_gradient)}
        snapshot_influence = np.asarray([snapshot_grad_map[k] for k in block_names_list])
        #Normalize:
        H_snapshot = np.sum(snapshot_influence)
        snapshot_influence = (Field_Budget/H_snapshot)*snapshot_influence
        
        
        con_names = ['no con','unif','full','block','sv','nc','snapshot']
        
        for s in tqdm.tqdm_notebook( range(Samples) )  : 

            initial_state = 1.0*np.ones(len(graph))
            no_control = Run_MonteCarlo_Average(relab_graph, T, beta_factor,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=background_field,initial_state=initial_state)

            initial_state = 1.0*np.ones(len(graph))
            unif_control_field = np.asarray([i+j for i,j in zip(uniform_control,background_field)])
            unif_control = Run_MonteCarlo_Average(relab_graph, T, beta_factor,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=unif_control_field,initial_state=initial_state)

            initial_state = 1.0*np.ones(len(graph))
            full_sus_control_field = np.asarray([i+j for i,j in zip(mag_grad_full_at_field_budget,background_field)])
            full_sus_control_mags = Run_MonteCarlo_Average(relab_graph, T, beta_factor,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=full_sus_control_field,initial_state=initial_state)

            initial_state = 1.0*np.ones(len(graph))
            block_sus_control_field = np.asarray([i+j for i,j in zip(mag_grad_block_at_field_budget,background_field)])
            block_sus_control_mags = Run_MonteCarlo_Average(relab_graph, T, beta_factor,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=block_sus_control_field ,initial_state=initial_state)

            initial_state = 1.0*np.ones(len(graph))
            sv_control_field = np.asarray([i+j for i,j in zip(swing_voter_control,background_field)])
            sv_control_mags = Run_MonteCarlo_Average(relab_graph, T, beta_factor,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=sv_control_field  ,initial_state=initial_state)

            initial_state = 1.0*np.ones(len(graph))
            nc_control_field = np.asarray([i+j for i,j in zip(negative_cancelling_field,background_field)])
            nc_control_mags = Run_MonteCarlo_Average(relab_graph, T, beta_factor,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=nc_control_field  ,initial_state=initial_state)

            initial_state = 1.0*np.ones(len(graph))
            snapshot_control_field = np.asarray([i+j for i,j in zip(snapshot_influence,background_field)])
            snapshot_control_mags = Run_MonteCarlo_Average(relab_graph, T, beta_factor,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=snapshot_control_field  ,initial_state=initial_state)

            
            means=[]
            ses=[]
            #Loop in same order as cont_names
            control_list = [no_control,unif_control,full_sus_control_mags,block_sus_control_mags,sv_control_mags,nc_control_mags,snapshot_control_mags]
            for k in control_list :
                print(mean_and_se(k))
                means.append(np.mean(k))
                ses.append(stats.sem(k))

            eval_data = eval_data.append(pd.DataFrame({'control':con_names,'magnetisation':means,'mag_se':ses,'beta_factor':beta_factor*np.ones(len(means)),'gradient':gradient*np.ones(len(means))}))
            eval_data.to_csv("Data/Pokec_control_eval_data_positive.csv")


/home/matt/anaconda3/lib/python3.7/site-packages/ising_block_level_influence-1.0-py3.7.egg/ising_block_level_influence/mean_field_IIM.py:169: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/home/matt/anaconda3/lib/python3.7/site-packages/ising_block_level_influence-1.0-py3.7.egg/ising_block_level_influence/mean_field_IIM.py:263: RuntimeWarning: divide by zero encountered in double_scalars



-0.6101971401527956 +/- 1.731473363552784e-05 
-0.586740795078088 +/- 1.628731923603448e-05 
-0.5571441079034548 +/- 7.451243619460779e-06 
-0.5809715773105266 +/- 7.200577262169681e-06 
-0.5885252450814683 +/- 3.8588673919476475e-05 
-0.5963123250625381 +/- 5.972855366713572e-06 
-0.5853442566425529 +/- 1.3913868019428313e-05 



-0.6106220066256507 +/- 1.9638857128895394e-05 
-0.5879648434859036 +/- 1.7769176022667178e-05 
-0.5669120613886823 +/- 1.0681459685840755e-05 
-0.5743910215671693 +/- 1.3294246103255496e-05 
-0.5870316002974781 +/- 1.709879385398948e-05 
-0.5974680346156447 +/- 1.381597662526939e-05 
-0.5759710837671557 +/- 1.8856026308263155e-05 



-0.6125559056182813 +/- 1.1897424443919949e-05 
-0.5844219390169697 +/- 1.3107642493311556e-05 
-0.5626842404164696 +/- 3.427034021839868e-05 
-0.5824350889054154 +/- 1.0902460207337155e-05 
-0.5831900141978231 +/- 1.2556500404921204e-05 
-0.5895544047055641 +/- 2.012164442930392e-05 
-0.584583645460077 +/- 1.0808902360218265e-05 



-0.6061355080792373 +/- 1.0569657379168216e-05 
-0.5796374957744574 +/- 1.1358337541563517e-05 
-0.554359346900142 +/- 8.140013565639355e-06 
-0.5684101413021432 +/- 1.696505328427251e-05 
-0.5899094449327293 +/- 9.985687613030656e-06 
-0.5876602055303901 +/- 7.619821141391628e-06 
-0.5806525049016293 +/- 1.1075732112769862e-05 


...


-0.18653477114461497 +/- 5.678476124645752e-06 
-0.17413565005746737 +/- 1.2721605807993752e-05 
-0.14446443107294976 +/- 8.068225603277082e-06 
-0.1583436752078967 +/- 6.085977297197132e-06 
-0.16293494692718546 +/- 1.6371449897385307e-05 
-0.17606655398553175 +/- 1.3935993061703549e-05 
-0.17103068081941725 +/- 9.688944339700446e-06 



-0.193859914813062 +/- 3.0153633861109706e-05 
-0.18110350212967347 +/- 2.0005978735492387e-05 
-0.137515225474951 +/- 1.652937431463726e-05 
-0.16667698600500302 +/- 1.117861823639952e-05 
-0.1623902711108106 +/- 8.0494648228659e-06 
-0.17003602866608072 +/- 1.4381063126353469e-05 
-0.16663022784125484 +/- 8.672687798212164e-06 



-0.18534579812047866 +/- 5.214300315613375e-06 
-0.1746159894530458 +/- 9.660657006728655e-06 
-0.14394817794604828 +/- 9.635257144845622e-06 
-0.1663366506659455 +/- 9.687172178582581e-06 
-0.16864281657764857 +/- 1.1098947005019206e-05 
-0.17207135420187952 +/- 6.846697464464734e-06 
-0.17163653573118787 +/- 4.461219053518458e-06 



-0.19709964843485903 +/- 6.854608018112312e-06 
-0.1830655601379217 +/- 7.951333814779821e-06 
-0.14396839970252182 +/- 1.0412840969526113e-05 
-0.16351321073625855 +/- 1.076273655112929e-05 
-0.1664259346900142 +/- 9.490300360611407e-06 
-0.17351799743087012 +/- 6.360494077295911e-06 
-0.17573738760056792 +/- 1.1660457678426023e-05 



-0.19101527956189576 +/- 1.831828686611937e-05 
-0.1763357311878845 +/- 1.2249569412843381e-05 
-0.13832770603745523 +/- 1.7060393221297052e-05 
-0.1745583733351362 +/- 1.4660805713623353e-05 
-0.16985393144479752 +/- 4.962868549046064e-06 
-0.17129462510986412 +/- 6.777713327611699e-06 
-0.16501700358326007 +/- 1.650933951160239e-05 



